<a href="https://colab.research.google.com/github/darwinj07/KoAlpaca-lora/blob/main/KoAlpaca_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLaMa-Alpaca-LoRA 파인튜닝

나만의 데이터로 ChatGPT 만들기

> 유튜브 [빵형의 개발도상국](https://www.youtube.com/@bbanghyong)

![](https://github.com/kairess/alpaca-lora/raw/main/result.jpg)

In [ ]:
!nvidia-smi

Mon Oct 16 10:49:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download source code and install dependencies

- https://github.com/tloen/alpaca-lora.git
- 주의!: bitsandbytes가 현재 Linux에서만 동작함
- 주의!!: peft 버그 https://github.com/tloen/alpaca-lora/issues/293

In [ ]:
!git clone -q https://github.com/darwinj07/KoAlpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

/content/alpaca-lora
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 

## Dataset

### - 49,620개의 Instruction, Input, Response set.

- https://github.com/Beomi/KoAlpaca/blob/main/ko_alpaca_data.json
- https://huggingface.co/datasets/Bingsu/ko_alpaca_data

### - shareGPT 한국어 번역 데이터셋

이 프로젝트는 shareGPT 데이터셋 60만 대화문을 DeepL 을 통해 한국어로 번역하고 있습니다.

https://huggingface.co/datasets/junelee/sharegpt_deepl_ko

```json
[
  {
    "instruction":"직원들의 급여를 기록하는 데이터베이스를 설계하십시오.",
    "input":"",
    "output":"직원들의 급여를 새로운 데이터베이스에서 추적하려면, 다음과 같은 열이 포함된 \"Employee\" 테이블을 만들어야 합니다: \"name\", \"salary\", \"start_date\", \"end_date\"."
  },
  {
    "instruction":"아래 문장의 각 단어에 품사를 지정하십시오.",
    "input":"새끼 고양이는 종종 신나게 뛰어다니기도 합니다.",
    "output":"새끼(Noun) 고양이(Noun) 는 은 종종 Adverb 신나게 Adverb 뛰어다니기도 합니다."
  },
  ...
]
```

## Finetune

기본 설정으로 했을 때:
- 3h/1epoch in NVIDIA A100
- 12h/1epoch in NVIDIA T4

### Create a custom template

Default Alpaca

```json
{
    "description": "Template used by Alpaca-LoRA.",
    "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
    "response_split": "### Response:"    
}
```

In [ ]:
import json

prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

### Hyperparameters

https://github.com/tloen/alpaca-lora/blob/0e1a5d52a460d14aea2325e43c302972badb9cdd/finetune.py#L28


In [ ]:
!python finetune.py \
    --base_model 'beomi/KoAlpaca-llama-1-7b' \
    --data_path 'instruction_updated_data.json' \
    --output_dir './output' \
    --num_epochs 7 \
    --learning_rate 5e-4 \
    --val_set_size 20 \
    --batch_size 8 \
    --micro_batch_size 8 \
    --prompt_template_name 'custom'
#use different path for new dataset and adjust parameters experimentally

2023-04-17 13:17:53.394813: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  war

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/KoAlpaca-LoRA
#or just manually download the files in './output'

MessageError: ignored

## Test on Gradio

--load_8bit

In [ ]:
!pip uninstall peft -y -q
!pip install peft
#you need to reinstall peft of newer version
!python generate.py \
    --load-8bit\
    --base_model 'beomi/KoAlpaca-llama-1-7b' \
    --lora_weights './output' \
    --prompt_template 'custom' \
    --share_gradio

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100% 3/3 [02:43<00:00, 54.40s/it]
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://3a492f7ca808618c07.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
/usr/local/lib/python3.10/dist-packages/gradio/components/button.py:89: Use